# F20DL Lab 8

## Setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=8)
mpl.rc('xtick', labelsize=8)
mpl.rc('ytick', labelsize=8)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
# Load the dataset into numpy arrays
images = np.load("./datasets/smiley_X.npy")
labels = np.load("./datasets/smiley_Y.npy")

FileNotFoundError: [Errno 2] No such file or directory: './datasets/smiley_X.npy'

In [ ]:
# Shows smiley as an image. 'smiley' should be an element in 'images'
def show_smiley(smiley):
    plt.figure(figsize=(2,2))
    plt.imshow(smiley, cmap="Greys")

In [ ]:
# Flatten image data and convert values to int
X = np.asarray([x.flatten() for x in images]).astype(int)
y = labels.astype(int)

In [ ]:
#Duplicate int Datasets
X2 = X.copy()
y2 = y.copy()
X3 = X.copy()
y3 = y.copy()

In [ ]:
# data is a list of all images. Images are represented as a flattened row of pixel values.
# To get pixel (r, c) of image i, use X[i-1][9*(r-1) + (c-1)] 
# Or use images[i][r-1][c-1][0]

# X[0] returns a row of pixel values representing the 1st image.
# X[0][24] and images[0][2][6][0] return the value of the pixel in the 3rd row and 7th column of the 1st image 
# (r=3, c=7).
# y[0] returns the label for the 1st image.
images[0][0][0][0]

## Decision Trees

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

# Tree 1

In [ ]:
# Create Decision Tree classifer object
classifier = DecisionTreeClassifier()

# Train Decision Tree Classifer
classifier = classifier.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = classifier.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test, y_pred)

# Tree 2

In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.6, random_state=1) # 40% training and 60% test

In [ ]:
# Create Decision Tree classifer object
classifier2 = DecisionTreeClassifier()

# Train Decision Tree Classifer
classifier2 = classifier2.fit(X2_train,y2_train)

#Predict the response for test dataset
y2_pred = classifier2.predict(X2_test)

In [ ]:
metrics.accuracy_score(y2_test, y2_pred)

# Tree 3

In [ ]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.9, random_state=1) # 10% training and 90% test

In [ ]:
# Create Decision Tree classifer object
classifier3 = DecisionTreeClassifier()

# Train Decision Tree Classifer
classifier3 = classifier3.fit(X3_train,y3_train)

#Predict the response for test dataset
y3_pred = classifier3.predict(X3_test)

## Random Forest Trees

In [ ]:
metrics.accuracy_score(y3_test, y3_pred)

# Tree 1

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFClassifier = RandomForestClassifier()

RFClassifier = RFClassifier.fit(X_train,y_train)

RFy_pred = RFClassifier.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test, RFy_pred)

# Tree 2

In [ ]:
RFClassifier2 = RandomForestClassifier()

RFClassifier2 = RFClassifier2.fit(X2_train,y2_train)

RFy2_pred = RFClassifier2.predict(X2_test)

In [ ]:
metrics.accuracy_score(y2_test, RFy2_pred)

# Tree 3

In [ ]:
RFClassifier3 = RandomForestClassifier()

RFClassifier3 = RFClassifier3.fit(X3_train,y3_train)

RFy3_pred = RFClassifier3.predict(X3_test)

In [ ]:
metrics.accuracy_score(y3_test, RFy3_pred)